In [ ]:
!pip install -q gradio huggingface_hub --upgrade
import gradio as gr
from huggingface_hub import InferenceClient


HF_TOKEN = "your own token"

client = InferenceClient(
    model="mistralai/Mistral-7B-Instruct-v0.3",
    token=HF_TOKEN
)

def is_safe_query(user_input):
    unsafe_keywords = [
        "suicide", "overdose", "self-harm", "kill", "die",
        "abort", "prescribe", "diagnose", "emergency"
    ]
    return not any(word in user_input.lower() for word in unsafe_keywords)

def chat_with_memory(message, history):
    history = history or []


    if not is_safe_query(message):
        history.append({"role": "user", "content": message})
        history.append({"role": "assistant", "content": "I'm sorry, but I can't help with that. Please consult a healthcare professional."})
        return history


    messages = [{"role": "system", "content": "You are a helpful and responsible medical assistant. Answer health questions clearly and carefully. Never give medical advice or prescriptions. Always recommend seeing a real doctor for serious issues."}]
    messages.extend(history)
    messages.append({"role": "user", "content": message})

    try:
        # Model inference
        response = client.chat_completion(messages, max_tokens=200, temperature=0.7)
        print("RAW RESPONSE:\n", response)

        if hasattr(response, "choices") and response.choices:
            choice = response.choices[0]
            if hasattr(choice, "message") and isinstance(choice.message, dict):
                reply = choice.message.get("content", "").strip()
            elif hasattr(choice, "text"):
                reply = choice.text.strip()
            else:
                reply = "Could not parse model response."
        else:
            reply = "Model did not return any choices."
    except Exception as e:
        reply = f"Exception: {str(e)}"

    # Append to history
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": reply})
    return history

# Launch Gradio ChatInterface
gr.ChatInterface(
    fn=chat_with_memory,
    chatbot=gr.Chatbot(type="messages"),
    title="General Health Chatbot",
    theme="default"
).launch(share=True, debug=True)


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://fe42fc2e18469bd95d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://fe42fc2e18469bd95d.gradio.live
